# Time Travel Debugger project

## Personal Information

In [1]:
PROJECT_TYPE = 2
NAME = ["Daniel Gusenburger", "Daniel Tabellion"]
ID = ["2544941", "2555742"]
IMPLEMENTED = set()

In [2]:
print(f"Implemented features: {IMPLEMENTED}")

Implemented features: set()


## Implementation

In [3]:
from time_travel_debugger.view.gui import GUI
from time_travel_debugger.view.cli import TimeTravelCLI
from main import remove_html_markup

    
with GUI():
    remove_html_markup("<tag>hallo</tag>")

Play(value=0)

IntSlider(value=1, description='Playback Speed', max=5, min=1)

ToggleButton(value=False, icon='history', tooltip='Reverse autoplay')

HTML(value='\n            <style>\n               .jupyter-widgets-output-area .output_scroll {\n             …

GridspecLayout(children=(HBox(children=(VBox(children=(Button(description='⯇', style=ButtonStyle()), Button(de…

...

TraitError: The 'step' trait of a Play instance must be an int, but a value of {'value': 3} <class 'dict'> was specified.

TraitError: The 'step' trait of a Play instance must be an int, but a value of {'value': 4} <class 'dict'> was specified.

TypeError: bad operand type for unary -: 'dict'

TypeError: bad operand type for unary -: 'dict'

TraitError: The 'step' trait of a Play instance must be an int, but a value of {'value': 1} <class 'dict'> was specified.

## Presentation

## Summary